In [ ]:
!pip install torch transformers underthesea tqdm pandas -q

In [ ]:
# ============================
# IMPORT THƯ VIỆN
# ============================
import pandas as pd                     # Xử lý dữ liệu dạng bảng (CSV)
import torch                            # Thư viện deep learning chính (PyTorch)
import torch.nn as nn                   # Mô-đun xây dựng mạng neural
from torch.utils.data import Dataset, DataLoader   # Dùng để quản lý và nạp dữ liệu huấn luyện
from transformers import AutoTokenizer, AutoModel  # Dùng để load PhoBERT (transformers của Hugging Face)
from tqdm import tqdm                   # Hiển thị thanh tiến trình khi train/predict
from underthesea import word_tokenize   # Thư viện tách từ tiếng Việt (rất quan trọng cho PhoBERT)

# ============================
# CẤU HÌNH CHUNG
# ============================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Nếu có GPU thì dùng, không thì fallback CPU
LABELS = ["giai_tri", "luu_tru", "nha_hang", "an_uong", "van_chuyen", "mua_sam"]  # 6 nhãn cảm xúc
BATCH_SIZE = 16                      # Số mẫu trong 1 batch (càng lớn thì train nhanh nhưng tốn VRAM)
EPOCHS = 100                        # Số vòng huấn luyện (có thể tăng lên 20–25 nếu GPU ổn)
LR = 2e-5                            # Learning rate – tốc độ học
MAX_LEN = 256                        # Chiều dài tối đa mỗi câu sau khi token hóa (PhoBERT giới hạn 256)

# ============================
# TIỀN XỬ LÝ DỮ LIỆU (Underthesea)
# ============================
def preprocess_text(text):
    """Tiền xử lý cơ bản + tách từ bằng underthesea"""
    text = str(text).lower().strip()                      # Chuyển thành chữ thường + bỏ khoảng trắng dư
    text = word_tokenize(text, format="text")             # Tách từ cho tiếng Việt ("học sinh" → "học_sinh")
    return text                                           # Trả về chuỗi đã xử lý

# ============================
# Dataset quản lý dữ liệu
# ============================
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=MAX_LEN):
        # Lưu danh sách câu đã qua xử lý
        self.texts = [preprocess_text(t) for t in texts]
        self.labels = labels                # Nhãn (6 giá trị cảm xúc)
        self.tokenizer = tokenizer          # PhoBERT tokenizer để chuyển text → token id
        self.max_len = max_len              # Chiều dài tối đa mỗi mẫu

    def __len__(self):
        return len(self.texts)              # Trả về số lượng mẫu dữ liệu

    def __getitem__(self, idx):
        text = self.texts[idx]              # Lấy câu thứ idx
        # Token hóa câu – chuyển văn bản thành input_ids & attention_mask
        inputs = self.tokenizer(
            text,
            padding="max_length",           # Đệm cho đủ max_length
            truncation=True,                # Cắt nếu quá dài
            max_length=self.max_len,
            return_tensors="pt"             # Trả về kiểu Tensor PyTorch
        )
        # Bỏ chiều batch 1 (squeeze) để DataLoader dễ ghép batch
        item = {k: v.squeeze(0) for k, v in inputs.items()}
        if self.labels is not None:
            # Nếu có nhãn, thêm vào batch (dạng torch.float để dùng loss)
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# ============================
# MÔ HÌNH PhoBERT + Neural Network
# ============================
class PhoBertNN(nn.Module):
    def __init__(self, model_name="vinai/phobert-base", hidden_size=768):
        super().__init__()
        self.phobert = AutoModel.from_pretrained(model_name)   # Load mô hình PhoBERT gốc
        # Mạng NN "đầu ra" xử lý vector CLS → 6 giá trị cảm xúc
        self.nn_head = nn.Sequential(
            nn.Dropout(0.1),               # Dropout giúp tránh overfitting
            nn.Linear(hidden_size, 512),   # Fully Connected: 768 → 512
            nn.ReLU(),                     # Hàm kích hoạt ReLU
            nn.Dropout(0.1),
            nn.Linear(512, 256),           # 512 → 256
            nn.ReLU(),
            nn.Linear(256, len(LABELS))    # 256 → 6 (số nhãn đầu ra)
        )

    def forward(self, input_ids, attention_mask):
        # Cho dữ liệu qua PhoBERT
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        # Lấy vector CLS (đại diện toàn câu) từ last_hidden_state
        cls_vec = outputs.last_hidden_state[:, 0]
        # Đưa qua NN head để dự đoán cảm xúc
        logits = self.nn_head(cls_vec)
        return logits                      # Trả về dự đoán (tensor kích thước [batch, 6])

# ============================
# HÀM HUẤN LUYỆN
# ============================
def train_epoch(model, loader, optimizer, criterion):
    model.train()                          # Đặt model ở chế độ huấn luyện
    total_loss = 0                         # Biến lưu tổng loss
    for batch in tqdm(loader, desc="Training"):  # Duyệt từng batch có thanh tiến trình
        optimizer.zero_grad()              # Reset gradient mỗi batch
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)

        outputs = model(input_ids, attention_mask)     # Forward
        loss = criterion(outputs, labels)              # Tính loss (MSE)
        loss.backward()                                # Lan truyền ngược
        optimizer.step()                               # Cập nhật trọng số
        total_loss += loss.item()                      # Cộng dồn loss

    return total_loss / len(loader)                    # Trả về loss trung bình mỗi epoch

# ============================
# HÀM DỰ ĐOÁN
# ============================
def predict(model, loader):
    model.eval()                                       # Chuyển sang chế độ dự đoán
    preds = []                                         # Lưu kết quả dự đoán
    with torch.no_grad():                              # Không tính gradient khi dự đoán
        for batch in tqdm(loader, desc="Predicting"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            outputs = model(input_ids, attention_mask)  # Kết quả đầu ra mô hình
            preds.append(outputs.cpu())                # Chuyển về CPU để lưu
    preds = torch.cat(preds, dim=0)                    # Ghép tất cả batch lại
    preds = torch.clamp(preds, 0, 5)                   # Giới hạn giá trị trong [0, 5]
    return preds.round().int().numpy()                 # Làm tròn và chuyển sang mảng numpy

# ============================
# CHƯƠNG TRÌNH CHÍNH
# ============================
def main():
    print("=== NẠP DỮ LIỆU ===")
    train_df = pd.read_csv("train_problem.csv")        # Đọc file train
    test_df = pd.read_csv("gt_reviews.csv")            # Đọc file test

    # Lấy cột văn bản và nhãn
    train_texts = train_df["Review"].fillna("").tolist()
    test_texts = test_df["review"].fillna("").tolist()
    y_train = train_df[LABELS].astype(float).values    # Chuyển nhãn sang dạng float (cho loss MSE)

    print("=== TẢI TOKENIZER PhoBERT ===")
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")  # Tải tokenizer PhoBERT

    # Tạo dataset và dataloader cho train/test
    train_ds = TextDataset(train_texts, y_train, tokenizer)
    test_ds = TextDataset(test_texts, None, tokenizer)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    print("=== KHỞI TẠO MÔ HÌNH PhoBERT + NN ===")
    model = PhoBertNN().to(DEVICE)                     # Load model lên GPU/CPU
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)  # Tối ưu hóa AdamW
    criterion = nn.MSELoss()                           # Dùng Mean Squared Error để đo sai số

    print("=== BẮT ĐẦU TRAIN ===")
    for epoch in range(EPOCHS):
        loss = train_epoch(model, train_loader, optimizer, criterion)
        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {loss:.4f}")

    print("=== DỰ ĐOÁN TRÊN TEST ===")
    preds = predict(model, test_loader)                # Dự đoán kết quả trên dữ liệu test

    # Lưu kết quả ra file CSV
    result = pd.DataFrame(preds, columns=LABELS)
    result.insert(0, "stt", range(1, len(result) + 1))
    result.to_csv("predictions.csv", index=False)

    print("\n✅ Đã tạo file predictions.csv:")
    print(result.head())                               # In vài dòng đầu để kiểm tra

# ============================
# CHẠY CHÍNH
# ============================
if __name__ == "__main__":
    main()  # Gọi hàm chính khi chạy file
